In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold
from optuna.visualization import plot_optimization_history

In [2]:
train=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

grouped = train.groupby("smoking")
df_0 = grouped.get_group(0)
df_1 = grouped.get_group(1)
df_0_half = df_0.sample(n=150, random_state=42)
df_1_half = df_1.sample(n=150, random_state=42)
df_half = pd.concat([df_0_half, df_1_half])

df_half = df_half.sample(frac=1, random_state=42).reset_index(drop=True)


In [3]:
train_copy=df_half.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test_new= test_set.drop('smoking', axis=1)
y_test_new = test_set['smoking']

In [4]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 8, 32, step=4)       
    n_a = trial.suggest_int("n_a", 8, 32, step=4)       
    n_steps = trial.suggest_int("n_steps", 3, 6)        
    gamma = trial.suggest_float("gamma", 1.0, 1.5)      
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-3, 5e-2, log=True)  
    max_epochs = trial.suggest_int("max_epochs", 20, 50)  

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        #  numpy
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=50,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            # eval_set=[(X_val_fold, y_val_fold)],
            # eval_metric=['accuracy'],
            max_epochs=max_epochs,
            # patience=5,
            batch_size=32,
            virtual_batch_size=32
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

print("best_params:", study.best_params)
print("best_value:", study.best_value)

[I 2025-08-22 15:18:09,789] A new study created in memory with name: no-name-079861bf-9a45-46f0-962a-cce7c648c51d
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.04213 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.96809 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 1.2305  |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.25041 |  0:00:00s


[I 2025-08-22 15:21:12,971] Trial 19 finished with value: 0.6966666666666665 and parameters: {'n_d': 28, 'n_a': 20, 'n_steps': 5, 'gamma': 1.4202103044536942, 'lambda_sparse': 8.624025055355895e-05, 'lr': 0.008105736247111157, 'max_epochs': 45}. Best is trial 11 with value: 0.7299999999999999.


best_params: {'n_d': 24, 'n_a': 20, 'n_steps': 6, 'gamma': 1.4975205131205425, 'lambda_sparse': 0.000149777536532999, 'lr': 0.011916250356066124, 'max_epochs': 42}
best_value: 0.7299999999999999


In [5]:
fig = plot_optimization_history(study)
fig.update_layout(
    title_text="TabNet Optimization Progress",  
    title_x=0.5,  
    title_font_size=20,  
    xaxis_title="Trial Number",  
    yaxis_title="Accuracy"
)
fig.show()